This file is a copied from a previous brackish water analysis file to update the baseline LCOW to the deep will injection scenarion in the Big Spring Case study

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import haversine as hs
import geopandas as gp
from shapely.geometry import Point, Polygon
import numpy as np

from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from pyomo.environ import Block, Expression, units as pyunits
import math
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
from truck_pipe_cost_functions import truck_costing, pipe_costing

1. Update deep well injection well depth to 2500 - DWI model updated to be a detailed model
2. Update to KBHDP treatment train
3. Minimum well yield - Updated to 0.02 m3/s
4. Well LCOW and antiscalant list to the table

In [23]:
case_study = 'kbhdp'
scenario = 'dwi'
desired_recovery = 1
ro_bounds = 'other'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

m.fs.ro_first_stage.membrane_area.unfix()
m.fs.ro_second_stage.membrane_area.unfix()
m = run_and_return_model(m, objective=True,print_it=True)

print('LCOW:', m.fs.costing.LCOW.value())


Case Study = KBHDP
Scenario = DWI


=========================ADDING UNIT PROCESSES=========================
WELL FIELD
MEDIA FILTRATION
CARTRIDGE FILTRATION
ANTI SCALANT ADDITION
RO FIRST STAGE
PASSTHROUGH
LIME SOFTENING
RO SECOND STAGE
CAUSTIC SODA ADDITION
CHLORINATION
TREATED STORAGE
DEEP WELL INJECTION
MUNICIPAL DRINKING


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 4

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 4

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

Case Study = KBHDP
Scenario = DWI


=========================ADDING UNIT PROCESSES=========================
WELL FIELD
MEDIA FILTRATION
CARTRIDGE FILTRATION
ANTI SCALANT ADDITION
RO FIRST STAGE
PASSTHROUGH
LIME SOFTENING
RO SECOND STAGE
CAUSTIC SODA ADDITION
CHLORINATION
TREATED STORAGE
DEEP WEL

In [26]:
# Read in brackish database. It is filtered for the correct columns. 
df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/USGS_brackish_filtered.csv', index_col='unique_site_ID')

# Convert USGS unit to our units: gpm to m3/s, 
df.well_yield = df.well_yield * 6.30902e-5 #gpm to m3/s
df.TDS_mgL = df.TDS_mgL / 1000 #mg/L to kg/m3
df = df.rename(columns={'TDS_mgL': 'TDS_kgm3'})

path = 'C:/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/data'

# Add state look up for electricity prices and add elec price column
state_abbv_df = pd.read_csv(path + '/state_abbv.csv', index_col='abbv')
elec_price_df = pd.read_csv(path + '/industrial_electricity_costs_2023.csv', index_col='location')
df['state'] = df.state_alpha.map(state_abbv_df.us_state)
df['elec_price'] = df.state.map(elec_price_df.cost)


# filter for well of interest in the US
df = df[df.well_yield >= 0.02] # Filter changed to 0.02 m3/s because after that pipe transport cost is > $1
df = df[df.TDS_kgm3 >= 0.5]
df = df[df.Latitude >= 22]
df = df[df.Longitude >= -150]
df = df[df.TDS_kgm3 <= 17] # Updated because treating anything higher results in very high pressures in second stage RO

# Loop through brackish sites and calculate+save LCOW, capital, O&M, energy intensity, and recovery in a df.
capital = []
recovery = []
onm = []
lcow = []
pipe_cost = []
elec_int = []
tds_out = []
flow_in_list = []
tds_in_list = []
well_field_annual_op_main_cost_list = []
well_field_total_cap_inv_list = []
well_field_lcow_list = []
i = 0


for site in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(i/len(df.index))

    try:
        # See if watertap3 solves without rerunning baseline case
        m.fs.ro_first_stage.membrane_area.unfix()
        m.fs.ro_second_stage.membrane_area.unfix()       
        
        m.fs.kbhdp_brackish_ave.flow_vol_in.fix(df.loc[site].well_yield)
        m.fs.kbhdp_brackish_ave.conc_mass_in[0, 'tds'].fix(df.loc[site].TDS_kgm3)
        m.fs.well_field.lift_height.fix(df.loc[site].well_depth_ft)
        
        m.fs.costing_param.electricity_price = df.loc[site].elec_price

        m = run_and_return_model(m, objective=True)
    
    except:
        m = watertap_setup(case_study=case_study, scenario=scenario)
        m = get_case_study(m=m)
        m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

        m.fs.ro_first_stage.membrane_area.unfix()
        m.fs.ro_second_stage.membrane_area.unfix()
        m.fs.ro_first_stage.feed.pressure.unfix()

        m.fs.kbhdp_brackish_ave.flow_vol_in.fix(df.loc[site].well_yield)
        m.fs.kbhdp_brackish_ave.conc_mass_in[0, 'tds'].fix(df.loc[site].TDS_kgm3)
        m.fs.well_field.lift_height.fix(df.loc[site].well_depth_ft)
        
        m.fs.costing_param.electricity_price = df.loc[site].elec_price
        m = run_and_return_model(m, objective=True,print_it=True)


    # Basis is recalculated to be on treated volume basis instead of brine volume
    temp = pipe_costing(m.fs.deep_well_injection.flow_vol_in[0].value*3600*24,16.0934,pumping_velocity = 1.5)*(1-m.fs.costing.system_recovery())/m.fs.costing.system_recovery()
    
    flow_in_list.append(df.loc[site].well_yield)
    capital.append(m.fs.costing.capital_investment_total())
    recovery.append(m.fs.costing.system_recovery()*100)
    onm.append(m.fs.costing.operating_cost_annual())
    pipe_cost.append(temp)
    lcow.append(m.fs.costing.LCOW())
    elec_int.append(m.fs.costing.electricity_intensity())
    tds_out.append(m.fs.municipal_drinking.conc_mass_in[0, 'tds']())
    tds_in_list.append(df.loc[site].TDS_kgm3)
    well_field_annual_op_main_cost_list.append(m.fs.well_field.costing.annual_op_main_cost())
    well_field_total_cap_inv_list.append(m.fs.well_field.costing.total_cap_investment())
    well_field_lcow_list.append(m.fs.well_field.LCOW())
    
df['treatment_lcow'] = lcow
df['pipe_lcow'] = pipe_cost
df['lcow'] = df['treatment_lcow'] + df['pipe_lcow'] # Total LCOW which includes treatment and brine disposal costs
df['recovery'] = recovery
df['capital'] = capital
df['onm'] = onm
df['elec_int'] = elec_int
df['tds_out'] = tds_out
df['well_field_o&m'] = well_field_annual_op_main_cost_list
df['well_field_cap_inv'] = well_field_total_cap_inv_list
df['well_field_lcow'] = well_field_lcow_list

-----------------------------
SITE NAME: AZ_GEOTH-AZ0183
0.00023752969121140142
.................................

Degrees of Freedom: 3

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0274
0.00047505938242280285
.................................

Degrees of Freedom: 3

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0297
0.0007125890736342043
.................................

Degrees of Freedom: 3

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0327
0.0009501187648456057
.................................

Degrees of Freedom: 3

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0338
0.0011876484560570072

Old Big Spring Run

In [1]:
case_study = 'big_spring'
scenario = 'dwi_a'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)

# run the baseline watertap3 version of this treatment train
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

m.fs.reverse_osmosis.membrane_area.unfix()
m.fs.reverse_osmosis.feed.pressure.unfix()

m = run_and_return_model(m, objective=True,print_it=True)

print(m.fs.costing.LCOW())


NameError: name 'watertap_setup' is not defined

In [3]:
# Read in brackish database. It is filtered for the correct columns. 
df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/USGS_brackish_filtered.csv', index_col='unique_site_ID')

# convert USGS unit to our units: gpm to m3s, 
df.well_yield = df.well_yield * 6.30902e-5 #gpm to m3s
df.TDS_mgL = df.TDS_mgL / 1000 #mgL to kg/m3
df = df.rename(columns={'TDS_mgL': 'TDS_kgm3'})

path = 'C:/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/data'

# add state look up for elec prices and add elec price column
state_abbv_df = pd.read_csv(path + '/state_abbv.csv', index_col='abbv')
elec_price_df = pd.read_csv(path + '/industrial_electricity_costs_2023.csv', index_col='location')
df['state'] = df.state_alpha.map(state_abbv_df.us_state)
df['elec_price'] = df.state.map(elec_price_df.cost)

# filter for well of interest in the US
df = df[df.well_yield >= 0.01] #---> to be updated
df = df[df.TDS_kgm3 >= 0.5]
df = df[df.Latitude >= 22]
df = df[df.Longitude >= -150]
df = df[df.TDS_kgm3 <= 25]

# loop through brackish sites and calculate+save LCOW, capital, O&M, energy intensity, and recovery in a df.
capital = []
recovery = []
onm = []
lcow = []
pipe_cost = []
elec_int = []
tds_out = []
flow_in_list = []
tds_in_list = []
well_field_annual_op_main_cost_list = []
well_field_total_cap_inv_list = []
well_field_lcow_list = []
i = 0

for site in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(i/len(df.index))
           
    m.fs.big_spring_feed.flow_vol_in.fix(df.loc[site].well_yield)
    m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(df.loc[site].TDS_kgm3)
    m.fs.well_field.lift_height.fix(df.loc[site].well_depth_ft)
    
    m.fs.costing_param.electricity_price = df.loc[site].elec_price

    m = run_and_return_model(m, objective=True)

    # Basis is recalculated to be on treated volume basis instead of brine volume
    temp = pipe_costing(m.fs.deep_well_injection.flow_vol_in[0].value*3600*24,16.0934,pumping_velocity = 1.5)*(1-m.fs.costing.system_recovery())/m.fs.costing.system_recovery()
    
    flow_in_list.append(df.loc[site].well_yield)
    capital.append(m.fs.costing.capital_investment_total())
    recovery.append(m.fs.costing.system_recovery()*100)
    onm.append(m.fs.costing.operating_cost_annual())
    pipe_cost.append(temp)
    lcow.append(m.fs.costing.LCOW())
    elec_int.append(m.fs.costing.electricity_intensity())
    tds_out.append(m.fs.municipal_drinking.conc_mass_in[0, 'tds']())
    tds_in_list.append(df.loc[site].TDS_kgm3)
    well_field_annual_op_main_cost_list.append(m.fs.well_field.costing.annual_op_main_cost())
    well_field_total_cap_inv_list.append(m.fs.well_field.costing.total_cap_investment())
    well_field_lcow_list.append(m.fs.well_field.LCOW())
    
df['treatment_lcow'] = lcow
df['pipe_lcow'] = pipe_cost
df['lcow'] = df['treatment_lcow'] + df['pipe_lcow'] # Total LCOW which includes treatment and brine disposal costs
df['recovery'] = recovery
df['capital'] = capital
df['onm'] = onm
df['elec_int'] = elec_int
df['tds_out'] = tds_out
df['well_field_o&m'] = well_field_annual_op_main_cost_list
df['well_field_cap_inv'] = well_field_total_cap_inv_list
df['well_field_lcow'] = well_field_lcow_list

-----------------------------
SITE NAME: AZ_GEOTH-AZ0183
0.0001953506544246923
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0274
0.0003907013088493846
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0297
0.000586051963274077
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0327
0.0007814026176987692
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0338
0.0009767532721234617
..

KeyboardInterrupt: 

In [27]:
# df.to_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_with_metrics_baseline_dwi_updated_costs_transport_updated_basis.csv') 
# df.to_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_baseline_dwi_5Feb23.csv')
# df.to_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_baseline_dwi_4Jun23.csv')